In [1]:
#pip install dash plotly
# pip install requests
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc, html

# Load the dataset
file_path = '../data/ZenithConnect_new.csv'
df = pd.read_csv(file_path)

# Set appropriate data types
df['customerID'] = df['customerID'].astype(str)
df['SeniorCitizen'] = df['SeniorCitizen'].astype(object)

# Create visualizations
fig_tenure = px.histogram(df, x='tenure', color='Churn', title='Distribution of Tenure by Churn')
fig_monthly_charges = px.histogram(df, x='MonthlyCharges', color='Churn', title='Distribution of Monthly Charges by Churn')
fig_senior_citizen = px.histogram(df, x='SeniorCitizen', color='Churn', title='Distribution of Senior Citizen by Churn')
fig_total_charges = px.histogram(df, x='TotalCharges', color='Churn', title='Distribution of Total Charges by Churn')
fig_gender = px.histogram(df, x='gender', color='Churn', title='Distribution of Gender by Churn')
fig_contract = px.histogram(df, x='Contract', color='Churn', title='Distribution of Contract by Churn')

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Customer Churn Analysis Dashboard"),
    dcc.Graph(figure=fig_tenure),
    dcc.Graph(figure=fig_monthly_charges),
    dcc.Graph(figure=fig_senior_citizen),
    dcc.Graph(figure=fig_total_charges),
    dcc.Graph(figure=fig_gender),
    dcc.Graph(figure=fig_contract),
])

# Save the Dash app as a static HTML file
def save_app_as_html(app, file_path):
    from dash import Dash
    from dash.development.base_component import Component
    from dash import dcc, html

    with open(file_path, 'w') as f:
        f.write('<!DOCTYPE html>\n<html>\n<head>\n<meta charset="UTF-8">\n<title>Customer Churn Analysis Dashboard</title>\n')
        f.write('<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>\n')
        f.write('</head>\n<body>\n')
        
        for component in app.layout.children:
            if isinstance(component, dcc.Graph):
                figure_html = component.figure.to_html(full_html=False, include_plotlyjs=False)
                f.write(figure_html)
        
        f.write('\n</body>\n</html>')

# Save the entire Dash app as a static HTML file
save_app_as_html(app, '../dashboards/ZenithCustomersDashboard.html')

if __name__ == '__main__':
    app.run_server(debug=False, port = 8070)

In [2]:
# Let's visualize on customer level esepcially those who who are going to churn

In [3]:
# Load datasets
file_path1 = '../data/ZenithConnect_new.csv'
file_path2 = '../data/model_results.csv'

df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

In [4]:
df = pd.merge(df1, df2, how = 'left', on = 'customerID')
df['ChurnScore'] = (df['Probability'] * 100).round(0).astype(int)

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import threading
import requests
import time
import plotly.subplots as sp


# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Customer Churn Analysis Dashboard"),
    dcc.Dropdown(
        id='customer-id-dropdown',
        options=[{'label': customer_id, 'value': customer_id} for customer_id in df['customerID'].unique()],
        placeholder="Select a Customer ID"
    ),
    html.Div(id='customer-info'),
    dcc.Graph(id='churn-score-graph')
])

import plotly.subplots as sp

@app.callback(
    [Output('customer-info', 'children'),
     Output('churn-score-graph', 'figure')],
    [Input('customer-id-dropdown', 'value')]
)
def update_dashboard(customer_id):
    if customer_id is None:
        return '', {}
    
    customer_data = df[df['customerID'] == customer_id].iloc[0]
    average_churn_score = df['ChurnScore'].mean()
    average_cltv = df['TotalCharges'].mean()
    
    info = html.Div([
        html.P(f"Customer ID: {customer_data['customerID']}"),
        html.P(f"Gender: {customer_data['gender']}"),
        html.P(f"Senior Citizen: {customer_data['SeniorCitizen']}"),
        html.P(f"Tenure: {customer_data['tenure']}"),
        html.P(f"Monthly Charges: {customer_data['MonthlyCharges']}"),
        html.P(f"CLTV: {customer_data['TotalCharges']}"),
        html.P(f"Contract: {customer_data['Contract']}"),
        html.P(f"Churn: {customer_data['Churn']}"),
        html.P(f"Predicted: {customer_data['Predicted']}"),
        html.P(f"Churn Score: {customer_data['ChurnScore']}")
    ])
    
    individual_score_color = 'green' if customer_data['ChurnScore'] < average_churn_score else 'red'
    individual_cltv_color = 'green' if customer_data['TotalCharges'] > average_cltv else 'red'
    
    # Create subplots
    fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Churn Score Comparison", "CLTV Comparison"))
    
    # Add churn score bars
    fig.add_trace(
        go.Bar(name='Churn Score', x=['Churn Score'], y=[customer_data['ChurnScore']], marker_color=individual_score_color),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(name='Average Churn Score', x=['Average Churn Score'], y=[average_churn_score], marker_color='blue'),
        row=1, col=1
    )
    
    # Add CLTV bars
    fig.add_trace(
        go.Bar(name='CLTV', x=['CLTV'], y=[customer_data['TotalCharges']], marker_color=individual_cltv_color),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(name='Average CLTV', x=['Average CLTV'], y=[average_cltv], marker_color='blue'),
        row=1, col=2
    )
    
    fig.update_layout(
        yaxis=dict(range=[0, 100]),
        yaxis2=dict(range=[0, max(df['TotalCharges'].max(), average_cltv)]),
        showlegend=False
    )
    
    return info, fig

def run_server():
    app.run_server(debug=False, port=8060)

if __name__ == '__main__':
    # Run the server in a separate thread
    threading.Thread(target=run_server).start()
    
    # Allow the server to start
    time.sleep(5)
    
    # Fetch the dashboard HTML
    url = "http://127.0.0.1:8060"
    response = requests.get(url)
    
    # Save the HTML to a file
    with open("../dashboards/ZenithPredictionsDashboard.html", "w") as file:
        file.write(response.text)


Rank customers based on their CLTV and probaility to churn .. this excel will go to the marketing team to contact these guys.

In [6]:
df['CLTV'] = df['TotalCharges'] 

df[df['Predicted'] == 1].sort_values(['CLTV', 'ChurnScore'], ascending=[False, False]).to_csv('../data/Valuable_PredictedToChurn.csv', index = False)